# Combining Python, SQL DBs and AWS

## Aim: Using python, create a DB on your AWS server, insert data into it and query that data to answer questions. 


# Using Python with MYSQL DB

In [1]:
## importing 'mysql.connector' 
import mysql.connector 
import datetime
import pandas as pd
from mysql.connector import errorcode

### Documentation:

https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html

## Using a config File:

You do not want to make your credentials viewable to everyone who might see this file.  So instead of explicity stating your credentials, we want to import them from another file.  

In [2]:
import config

In [3]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    password = config.password
)

print(cnx) # it will print a connection object if everything is fine


cursor = cnx.cursor()

## Creating DB

In [4]:
db_name = 'students'

Function to help us create a database

In [5]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [6]:
# check to see if the database already 
try:
    cursor.execute("USE {}".format(db_name))

#if the previous line fails because there isn't a db by that name run this line

except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

### Creating tables

In [4]:
DB_NAME = 'employees'

TABLES = {}
TABLES['employees'] = (
    "CREATE TABLE employees ("
    "  emp_no int(11) NOT NULL AUTO_INCREMENT,"
    "  birth_date date NOT NULL,"
    "  first_name varchar(14) NOT NULL,"
    "  last_name varchar(16) NOT NULL,"
    "  gender enum('M','F') NOT NULL,"
    "  hire_date date NOT NULL,"
    "  PRIMARY KEY (emp_no)"
    ") ENGINE=InnoDB")

TABLES['departments'] = (
    "CREATE TABLE departments ("
    "  dept_no char(4) NOT NULL,"
    "  dept_name varchar(40) NOT NULL,"
    "  PRIMARY KEY (dept_no), UNIQUE KEY dept_name (dept_name)"
    ") ENGINE=InnoDB")

TABLES['salaries'] = (
    "CREATE TABLE salaries ("
    "  emp_no int(11) NOT NULL,"
    "  salary int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT salaries_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_emp'] = (
    "CREATE TABLE dept_emp ("
    "  emp_no int(11) NOT NULL,"
    "  dept_no char(4) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no), KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_emp_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_emp_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_manager'] = (
    "  CREATE TABLE dept_manager ("
    "  dept_no char(4) NOT NULL,"
    "  emp_no int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no),"
    "  KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_manager_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_manager_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['titles'] = (
    "CREATE TABLE titles ("
    "  emp_no int(11) NOT NULL,"
    "  title varchar(50) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date DEFAULT NULL,"
    "  PRIMARY KEY (emp_no,title,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT titles_ibfk_1 FOREIGN KEY (emp_no)"
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

In [8]:
TABLES['employees']

"CREATE TABLE employees (  emp_no int(11) NOT NULL AUTO_INCREMENT,  birth_date date NOT NULL,  first_name varchar(14) NOT NULL,  last_name varchar(16) NOT NULL,  gender enum('M','F') NOT NULL,  hire_date date NOT NULL,  PRIMARY KEY (emp_no)) ENGINE=InnoDB"

In [9]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

In [12]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table employees: already exists.
Creating table departments: already exists.
Creating table salaries: already exists.
Creating table dept_emp: already exists.
Creating table dept_manager: already exists.
Creating table titles: already exists.


## Inserting Data

In [46]:
from datetime import date, datetime, timedelta

#since we closed the connection, we need to reinstatiate it
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = db_name
)
cursor = cnx.cursor()

In [47]:
tomorrow = datetime.now().date() + timedelta(days=1)

In [48]:
# insert statement to add an employee
add_employee = ("INSERT INTO employees "
               "(first_name, last_name, hire_date, gender, birth_date) "
               "VALUES (%s, %s, %s, %s, %s)")

#insert statement to add a salary
add_salary = ("INSERT INTO salaries "
              "(emp_no, salary, from_date, to_date) "
              "VALUES (%(emp_no)s, %(salary)s, %(from_date)s, %(to_date)s)")

#data to insert for one employee
data_employee = ('Geert', 'Vanderkelen', tomorrow, 'M', date(1977, 6, 14))



In [49]:
# Insert new employee
cursor.execute(add_employee, data_employee)
# Make sure data is committed to the database
cnx.commit()

ProgrammingError: 1146 (42S02): Table 'students.employees' doesn't exist

In [50]:
# grabs the id of the employee we just inserted so we can use it to input salary data
emp_no = cursor.lastrowid

In [51]:

# create salary information
data_salary = {
  'emp_no': emp_no,
  'salary': 50000,
  'from_date': tomorrow,
  'to_date': date(9999, 1, 1),
}

In [52]:

#Insert that salary information
cursor.execute(add_salary, data_salary)

# Make sure data is committed to the database
cnx.commit()

ProgrammingError: 1146 (42S02): Table 'students.salaries' doesn't exist

## Insert Many

If you have a list of records that you would like to input, there are two ways you can do this.  YOu can loop over each record in python and execute each record, or you can use the `executemany()` method.  

`cursor.executemany(operation, list_of_params)`



In [53]:
#create a list of records

data = [
  ('Jane', date(2005, 2, 12)),
  ('Joe', date(2006, 5, 23)),
  ('John', date(2010, 10, 3)),
]

In [54]:
#create the insert statment
stmt = "INSERT INTO employees (first_name, hire_date) VALUES (%s, %s)"


In [55]:
#insert all of the records and commit it
cursor.executemany(stmt, data)
cnx.commit()

ProgrammingError: 1146 (42S02): Table 'students.employees' doesn't exist

In [ ]:
#Close the connection 

cursor.close()
cnx.close()

## Quyerying the DB

In [4]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    #database = 'students'
    database = 'employees'
)
cursor = cnx.cursor()

In [47]:

query = """
SELECT 
    first_name, 
    last_name, 
    hire_date 
FROM 
    employees
    ;"""

hire_start = datetime.date(2006, 1, 1)
hire_end = datetime.date(2006, 12, 31)

cursor.execute(query)



In [48]:
result = cursor.fetchall()

In [49]:
df = pd.DataFrame(result)
df.columns = [x[0] for x in cursor.description]
df

,first_name,last_name,hire_date
0,Geert,Vanderkelen,2019-12-04
1,Jane,,2005-02-12
2,Joe,,2006-05-23
3,John,,2010-10-03
4,Geert,Vanderkelen,2019-12-04
5,Geert,Vanderkelen,2019-12-05
6,Geert,Vanderkelen,2019-12-05
7,Geert,Vanderkelen,2019-12-05


In [24]:
config.make_df(cursor, query)

,first_name,last_name,hire_date
0,Geert,Vanderkelen,2019-12-04
1,Jane,,2005-02-12
2,Joe,,2006-05-23
3,John,,2010-10-03
4,Geert,Vanderkelen,2019-12-04
5,Geert,Vanderkelen,2019-12-05
6,Geert,Vanderkelen,2019-12-05
7,Geert,Vanderkelen,2019-12-05


### How to read the data from your query.

You can use the `fetchone()`, `fetchmany()`, or `fetchall()` methods.
 


The `fetchone()` method returns the next row of a query result set or `None` in case there is no row left.

        row = cursor.fetchone()
 
        while row is not None:
            print(row)
            row = cursor.fetchone()

In case the number of rows in the table is small, you can use the `fetchall()` method to fetch all rows from the database table.

        rows = cursor.fetchall()
 
        print('Total Row(s):', cursor.rowcount)
        for row in rows:
            print(row)

**Why might you use a fetchone() when you have a query that will return multiple rows?**

For a relatively big table, it takes time to fetch all rows and return the entire result set. In addition, `fetchall()` needs to allocate enough memory to store the entire result set in the memory, which is not efficient.

MySQL Connector/Python has the `fetchmany()` method that returns the next number of rows (n) of the result set, which allows you to balance between retrieval time and memory space.



First, develop a generator that chunks the database calls into a series of `fetchmany()` calls:

In [ ]:
def iter_row(cursor, size=10):
    while True:
        rows = cursor.fetchmany(size)
        if not rows:
            break
        for row in rows:
            yield row

Second, use the  iter_row() generator to fetch 10 rows at a time :

`cursor.execute("SELECT * FROM books")`
 
`for row in iter_row(cursor, 10):
    print(row)`

In [ ]:
#in practice

data = cursor.fetchall()

In [ ]:
print(data)

In [ ]:
cursor.close()
cnx.close()

## Applied: Working with our student data

### Import our student data

In [38]:
import json

f=open('students.json','r')
data=json.load(f)

In [39]:
#examine that student data
data[0].keys()

dict_keys(['name', 'dob', 'fav_food', 'birthplace', 'years_in_nyc'])

In [40]:
type(data)

list

### Determine how you should define your table to hold the data


cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    password = config.password
)

print(cnx) # it will print a connection object if everything is fine


cursor = cnx.cursor()

In [10]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    password = config.password
)

c = cnx.cursor()

cnx.database = 'students'

### Create a table for our student info

In [148]:
add_student = ("INSERT INTO students "
               "(name, dob, fav_food, birthplace, years_in_nyc) "
               "VALUES (%s, %s, %s, %s, %s);")

In [149]:

create_query =""" 
CREATE TABLE students (
    name varchar(100) NOT NULL, 
    dob varchar(50) NOT NULL, 
    fav_food varchar(100) NOT NULL, 
    birthplace varchar(100) NOT NULL, 
    years_in_nyc float
);
"""

### Now that we have created our table we need to insert our data into the table.

We can't just pass the execute query a list of dictioanries,  We need to parse through it first. 

Take one student and identify how you would parse that information into a usable format.

In [152]:
test_student = data[-1]

In [157]:
students_list

[('Anastasia Gorina', '1991-07-19', 'oysters', 'St. Petersburg, Russia', 5),
 ('Andres M Chaves', '1983-01-22', 'chocolate', 'Bogota, CU', 6),
 ('Andrew Triola', '1989-10-17', 'baked ziti', 'Washington DC', 3),
 ('Anita Guo', '1990-12-17', 'Seafood Boil', 'Fuzhou, China', 16),
 ('Askhat Yktybaev', '1981-25-02', 'cheese cake', 'Bishkek, Kyrgyzstan', 3),
 ('Aviva Sally Mazurek', '1994-04-29', 'Burgers', 'Bronx, NY', 2),
 ('Ben Johnson-Laird',
  '1900-04-01',
  'Roast Beef and Yorkshire pudding',
  'London, UK',
  21),
 ('Brittany Fowle', '1992-05-27', 'noodles', 'Painesville, OH', 9),
 ('Chadwick Balloo', '1991-07-29', 'chicken', 'Queens, New York', 1),
 ('Ed Haracz', '1995-02-24', 'pizza', 'Glen Ridge, NJ', 0),
 ('Grace Park', '1990-04-05', 'sushi', 'Flushing, NY', 29),
 ('Jackson Bull', '1987-11-28', 'fried_chicken', 'Nashville, TN', 8),
 ('Julia Chong', '1996-11-27', 'soup dumplings', 'Portland, OR', 1.5),
 ('Owen Peng', '1996-05-04', 'Burgers', 'OKC, OK', 13),
 (' Raphael Krantz', '1

Now use a for loop to loop over all of data and return a list that we can use for `executemany()`.

**If all of your data isn't formatted as expected, this will cause your loop to fail**  you might have to account for 'bad data' in your for loop.

In [155]:
students_list = []
    
for student in data:

    student_tuple= (student['name'], student['dob'],student['fav_food'],student['birthplace'],student['years_in_nyc'])
    students_list.append(student_tuple)

Now that we have this list of tuples, lets use `executemany()` to insert the data.

In [172]:
# Insert new employee
c.executemany(add_student, students_list)
# Make sure data is committed to the database
cnx.commit()

Now that we have the data in the database, we can write

### Write queries to answer the following questions:

**Questions**
- Which student was born closest to the cohort's graduation date?
- Which student has the most siblings?
- How many students are only children?
- Which 3 students have lived in NYC the shortest amount of time?
- How many students are native New Yorkers?
- Do any two students have the same favorite food?

In [160]:
q1 = """
SELECT name
FROM students
ORDER BY years_in_nyc ASC
LIMIT 3
"""

In [161]:
config.make_df(c, q1)

,name
0,Ed Haracz
1,Chadwick Balloo
2,Julia Chong


In [5]:
q2 ="""
SELECT *
FROM students
"""

In [6]:
df2 = config.make_df(c, q2)

AttributeError: module 'config' has no attribute 'make_df'

In [131]:
df2[df2['fav_food'].duplicated(keep=False)]
    

,name,dob,fav_food,birthplace,years_in_nyc
0,Sean Abu Wilson,1985-02-06,strawberry cake,"Birmingham, AL",8.0
6,Aviva Sally Mazurek,1994-04-29,Burgers,"Bronx, NY",2.0
14,Owen Peng,1996-05-04,Burgers,"OKC, OK",13.0
18,Sean Abu Wilson,1985-02-06,strawberry cake,"Birmingham, AL",8.0


In [132]:
df2[df2['fav_food'].duplicated() == True]

,name,dob,fav_food,birthplace,years_in_nyc
14,Owen Peng,1996-05-04,Burgers,"OKC, OK",13.0
18,Sean Abu Wilson,1985-02-06,strawberry cake,"Birmingham, AL",8.0


In [164]:
df2.duplicated(subset='fav_food', keep='last')

0     False
1     False
2     False
3     False
4     False
5      True
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
dtype: bool

In [165]:
seen = set()
uniq = []
duplicates = []
for x in df2['fav_food']:
    if x not in seen:
        uniq.append(x)
        seen.add(x)
    else:
        duplicates.append(x)   

In [166]:
duplicates

['Burgers']

In [167]:
df2[df2['fav_food'].isin(duplicates)]

,name,dob,fav_food,birthplace,years_in_nyc
5,Aviva Sally Mazurek,1994-04-29,Burgers,"Bronx, NY",2.0
13,Owen Peng,1996-05-04,Burgers,"OKC, OK",13.0


In [168]:
q3 = """
SELECT 
    a.*
FROM 
    students.students a
JOIN (SELECT name, fav_food, COUNT(*)
FROM 
    students.students
GROUP BY 
    fav_food
HAVING 
    count(*) > 1 ) b
ON 
    a.fav_food = b.fav_food
;
"""

In [169]:
config.make_df(c, q3)

,name,dob,fav_food,birthplace,years_in_nyc
0,Aviva Sally Mazurek,1994-04-29,Burgers,"Bronx, NY",2.0
1,Grace Park,1990-04-05,sushi,"Flushing, NY",29.0
2,Owen Peng,1996-05-04,Burgers,"OKC, OK",13.0
3,Rob Zifchak,1989-08-02,Sushi,"Santa Monica, CA",1.5


In [170]:
q4 = """

DELETE t1 FROM students.students t1
INNER JOIN students.students t2 
WHERE 
    t1.name = t2.name;
"""

In [171]:
c.execute(q4)
cnx.commit()

In [11]:
q5 = """
   SET dob = STR_TO_DATE(dob, '%Y-%m-%d') ;
"""

In [12]:
c.execute(q5)

DatabaseError: 1193 (HY000): Unknown system variable 'dob'